Code adapted from example provided in: https://www.datacamp.com/tutorial/decision-tree-classification-python

In [ ]:
!pip install skl2onnx

In [ ]:
import pandas as pd
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn import metrics
from skl2onnx import to_onnx
from google.colab import drive

drive.mount('/content/drive/')
path = 'replace with path to folder with data files/' #Defining the path

Mounted at /content/drive/


In [ ]:
col_names = ['CH1','LABEL']
csvData = pd.read_csv(path+"SensorData.csv", header=None, names=col_names)

In [ ]:
sensors = ['CH1']  # The channel containing sensor data, i.e. the independent variables
X = csvData[sensors] # The independent variables (i.e. the EMG signals) from each channel that reflect either rest or muscle activation
Y = csvData.LABEL # The dependent variable, which states whether the corresponding data is 'positive' or 'negative'
# Splits the data into training and testing groups
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2) # 80% training and 20% test

In [ ]:
clf = svm.SVC() # Creates the SVM classifier object
clf.fit(X_train, Y_train) # Trains the classifier on the training dataset

# This block of code saves the model to an ONNX file in the same drive folder as the data files.
# Code based on documentation example:  https://onnx.ai/sklearn-onnx/index.html
onx = to_onnx(clf, X[:1])
with open(path+"svm.onnx", "wb") as f:
    f.write(onx.SerializeToString())

# Tests the SVM classifier on the test dataset
Y_pred = clf.predict(X_test)

In [ ]:
cf = confusion_matrix(Y_test, Y_pred, labels=clf.classes_)  # Creates a confusion matrix object using the test results
conf_display = ConfusionMatrixDisplay(confusion_matrix = cf, display_labels = clf.classes_) # Loads the confusion matrix into a display

print("Accuracy:",metrics.accuracy_score(Y_test, Y_pred)) # Prints the accuracy of the model as tested against the test dataset

conf_display.plot() # Plots the confusion matrix